In [ ]:
from morphology import computeMorphologyMetrics
import os
from datetime import datetime
from cv2 import cv2 as cv
from matplotlib import pyplot as plt
%matplotlib inline

# Computes morphology metrics and writes out an xlsx file with the numerical results.
# NOTE: Results will be written to a sub directory of where the file/s analyzed are located.
#
# search_image_path:            Path to the image or directory of images that metrics are to be computed for.
#                               OPTIONS:
#                               - "select_file" or "select_dir' (case is ignored),
#                                 a small window will pop up allowing selection of a file or directory 
#                                 from the file system where the image or images exist.
#
# template_image_paths:         A list of 2 paths to templates that will be located in search_image. 
#                               i.e. template_image_paths = [path_to_template_1_image, path_to_template_2_image]
#                               These templates should have inner edges that form vertical end points of
#                               a sub region in search_image that will be analyzed i.e.
#                               the left template should have it's right vertical edge as the
#                               left most point in search_image to consider, and 
#                               the right template should have it's left vertical edge as the 
#                               right most point in search_image to consider.
#                               OPTIONS: 
#                               - "select_files" (case is ignored),
#                                 a window will pop up allowing selection from the file system.
#                               - "draw_rois", for every image being analyzed,
#                                 a window will pop up where you can manually select ROIs.
#                                 To draw the ROIs, the process is as follows:
#                                 draw the bounding box for an roi and then hit enter, repeat for every roi,
#                                 when all ROIs have been drawn, hit ESC to complete the selection.
#                               - "draw_rois_once", the procedure is the same as for "draw_rois",
#                                 but only the first image will appear to select ROIs from
#                                 all other images to analyze will use these same templates.

template_image_paths = 'draw_rois'
directory_with_images = '/storage/data/home/positimothy/dev/repo/curibio/optical-tracking/test_data/morphology/multi_test/'
computeMorphologyMetrics(
    search_image_path=directory_with_images,
    template_image_paths=template_image_paths,
    sub_pixel_search_increment=None,
    sub_pixel_refinement_radius=None,
    microns_per_pixel=1.2,
    write_result_images=False,
    display_result_images=True    
)